In [5]:
import datetime
import urllib

from urlparse import urlparse
from threading import Thread
import httplib, sys
from Queue import Queue

## Generate list of XLS URLs
https://www.misoenergy.org/RTDisplays/MKTRPT/AllReportsList.html?rptName=Daily%20Regional%20Forecast%20and%20Actual%20Load%20(xls)# 

Website loads the list of .xls files using javascript, so `urllib2` won't see them. Rather than using a headless webkit browser, generating the URLs will be easier since the URL signature is fixed for the download links of all .csv files.

In [6]:
# Generate date range that matches date signature in URL of .xls files
endDate = datetime.datetime.strptime('20130201', '%Y%m%d')
startDate = datetime.datetime.strptime('20130101', '%Y%m%d')

dayDelta = (endDate-startDate).days
dateList = [ startDate + datetime.timedelta(days=n) for n in range(0, dayDelta) ]
dateList = [datetime.datetime.strftime(d, '%Y%m%d') for d in dateList]

In [7]:
# Generate URL list (for checking, will not be used directly)
urlList = ['https://www.misoenergy.org/Library/Repository/Market%20Reports/' + d + '_rf_al.xls' for d in dateList]

## Download XLS files from MISO
Download files concurrently to increase download throughput

In [10]:
concurrent = 10

def doWork():
    while True:
        d = q.get()
        url = 'https://www.misoenergy.org/Library/Repository/Market%20Reports/' + d + '_rf_al.xls'
        csvFile = urllib.URLopener()
        csvFile.retrieve(url, '../miso/' + d + '_rf_al.xls')

In [ ]:
%%time

q = Queue(concurrent * 2)

for i in range(concurrent):
    t = Thread(target=doWork)
    t.daemon = True
    t.start()
    
try:
    for date in dateList:
        q.put(date)
    q.join()
except KeyboardInterrupt:
    sys.exit(1)